In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
# Import pyMC3 and also arviz for visualisation
import pymc as pm
import arviz as az
import sympy as sp
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('bmh')

RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)

In [ ]:
fy,yuud,ytd, u, T, delta = sp.symbols('fy,Y_uud,Y_td, u, T, delta')
eq_fy = sp.Eq(fy,yuud*u**2*delta + ytd*T*delta)
eq_fy

In [ ]:
lambda_fy = sp.lambdify(list(eq_fy.rhs.free_symbols), eq_fy.rhs)

## Data

In [ ]:
# Size of dataset
size = 20

data = pd.DataFrame()
# Predictor variable
data['delta'] = np.linspace(0,np.deg2rad(10),size)
data['u'] = 0.1
data['T'] = 0.1

# Simulate outcome variable
ytd_=0.5
yuud_=0.2

data['fy'] = lambda_fy(Y_uud=yuud_, Y_td=ytd_, u=data['u'], delta=data['delta'], T=data['T'])

sigma_=0.2*data['fy'].abs().max()
data['fy_measure'] = data['fy'] + rng.normal(size=size) * sigma_

In [ ]:
variables = [u,T,delta]
features = eq_fy.rhs.free_symbols - set(variables)
A_,b_ = sp.linear_eq_to_matrix(eq_fy, features)
A_

In [ ]:
features

In [ ]:
lambda_features = sp.lambdify(list(A_.free_symbols),A_)
lambda_labels = sp.lambdify(list(b_.free_symbols),b_)

In [ ]:
columns =[symbol.name for symbol in features]
X = pd.DataFrame(lambda_features(T=data['T'], delta=data['delta'], u=data['u'])[0].T, columns=columns, index=data.index)

In [ ]:
y = pd.Series(lambda_labels(data['fy']).flatten(), index=data.index)

In [ ]:
features

In [ ]:
columns

In [ ]:
mus = {
    'Y_uud':yuud_,
    'Y_td':ytd_,
}

stds = {
    'Y_uud':0.02,
    'Y_td':0.02,
}



In [ ]:
basic_model = pm.Model()

sigmas_ = [stds[key] for key in columns]
mus_ = [mus[key] for key in columns]

with basic_model:
    # Priors for unknown model parameters
    parameters = pm.Normal("parameters", mu=mus_, 
                           sigma=sigmas_, 
                           shape=len(mus))
    
    sigma = pm.HalfNormal("sigma", sigma=sigma_)

    # Expected value of outcome
    mu = pm.math.dot(X, parameters)

    # Likelihood (sampling distribution) of observations
    fy_obs = pm.Normal("fy_obs", mu=mu, sigma=sigma, observed=data['fy_measure'])

In [ ]:
with basic_model:
    # draw 1000 posterior samples
    trace = pm.sample(draws=100)

In [ ]:
with basic_model:
    az.plot_posterior(trace,
                  var_names=['parameters','sigma'],
                  textsize=18,
                  point_estimate='mean',
                  rope_color='black')

In [ ]:
az.summary(trace)

In [ ]:
means = (trace.posterior['parameters'].data[0].mean(axis=0) + trace.posterior['parameters'].data[1].mean(axis=0))/2
means = {key:mean for key,mean in zip(columns,means)}

In [ ]:
data['Bayesian'] = lambda_fy(Y_uud=means['Y_uud'], Y_td=means['Y_td'], u=data['u'], delta=data['delta'], T=data['T'])

In [ ]:
fig,ax=plt.subplots()
data.plot(x='delta', y='fy', ax=ax)
data.plot(x='delta', y='fy_measure', style='.', ax=ax)
data.plot(x='delta', y='Bayesian', style='--', ax=ax)

for params in trace.posterior['parameters'][0].data:
    param = {key:mean for key,mean in zip(columns,params)}
    ax.plot(data['delta'],lambda_fy(Y_uud=param['Y_uud'], Y_td=param['Y_td'], u=data['u'], delta=data['delta'], T=data['T']), alpha=0.1, color='grey', zorder=-10)